<a href="https://colab.research.google.com/github/heron-ho/Imersao_IA_heron/blob/main/avaliacao_imersao_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### SDK

In [7]:
!pip install -q -U google-generativeai

### Importações

In [8]:
## importando bibliotecas
import google.generativeai as genai
import pandas as pd
import numpy as np
from google.colab import userdata

### Autenticação e configuração da api

In [9]:

API_KEY = userdata.get('secret_key')
#API_KEY = ... (está no secret key)
genai.configure(api_key=API_KEY)


### Configurações do modelo de IA generativa
#### Será usado na geração de uma mensagem para o aluno.

In [118]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

generation_config = {
    'candidate_count': 1, # pedidos podem dar mais de uma resposta/alternativa
    'temperature':  0.2,
}

safety_settings = { #precisa de ajustes
    'HARASSMENT': 'BLOCK_NONE',
    'HATE':'BLOCK_NONE',
    'SEXUAL':'BLOCK_NONE',
    'DANGEROUS':'BLOCK_NONE'
}



models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


###Começo da interação (testes)


In [117]:
# em processo....
json = input('forneça os dados sobre os alunos em JSON: ')
area = input('forneça as palavras relacionadas à área do conhecimento da qual você quer encontrar um destaque')

forneça os dados sobre os alunos em JSON: teste
forneça as palavras relacionadas à área do conhecimento da qual você quer encontrar um destaqueteste


### Gerando variável para modelo generativo de IA

In [120]:
modelo = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config = generation_config,
                              safety_settings = safety_settings)

###Recebendo dados em JSON
por hora via célula

In [121]:
info_alunos = [
  {
    "nome": "Ana Silva",
    "autoavaliacao": "No semestre passado, pude observar um crescimento significativo em minhas habilidades de comunicação e trabalho em equipe. Participei ativamente de projetos em grupo, aprimorando minha capacidade de colaborar com colegas e apresentar ideias de forma clara e concisa. Além disso, desenvolvi um maior interesse em áreas como estatística e análise de dados, o que me motivou a buscar cursos extracurriculares para aprofundar meus conhecimentos.",
  },
  {
    "nome": "Bruno Oliveira",
    "autoavaliacao": "Ao longo do semestre, notei um progresso considerável em minhas habilidades de programação e resolução de problemas. Dediquei-me bastante à prática de codificação, participando de desafios e competindo com colegas. Também me interessei por áreas como machine learning e inteligência artificial, o que me levou a iniciar pesquisas independentes sobre o assunto.",
  },
  {
    "nome": "Carla Costa",
    "autoavaliacao": "Neste semestre, tive a oportunidade de aprimorar minhas habilidades de escrita e pesquisa. Participei de diversos trabalhos escritos, desde resumos até artigos científicos, o que me permitiu desenvolver uma melhor compreensão da estrutura textual e da organização de ideias. Além disso, me interessei cada vez mais pela área de literatura, o que me motivou a ler obras de autores renomados e participar de grupos de discussão literária.",
  },
  {
    "nome": "Daniel Santos",
    "autoavaliacao": "Durante o semestre, pude observar um crescimento significativo em minhas habilidades de gestão de tempo e organização. Aprendi a priorizar tarefas, utilizar ferramentas de planejamento e otimizar meu tempo de estudo. Além disso, me interessei por áreas como administração e gestão de projetos, o que me levou a buscar cursos extracurriculares para aprimorar minhas habilidades nesse campo.",
  },
  {
    "nome": "Erika Pereira",
    "autoavaliacao": "No semestre passado, tive a oportunidade de desenvolver minhas habilidades de liderança e trabalho em equipe. Participei ativamente de projetos em grupo, assumindo responsabilidades e orientando meus colegas. Além disso, me interessei cada vez mais pela área de psicologia, o que me motivou a buscar cursos extracurriculares para aprofundar meus conhecimentos sobre comportamento humano.",
  }
]

#### Alternativa ao formato JSON (EXCEL)

In [122]:
# Cógigos aqui

### Montar dataframe

In [123]:
df = pd.DataFrame(info_alunos)
display(df)

,nome,autoavaliacao
0,Ana Silva,"No semestre passado, pude observar um crescime..."
1,Bruno Oliveira,"Ao longo do semestre, notei um progresso consi..."
2,Carla Costa,"Neste semestre, tive a oportunidade de aprimor..."
3,Daniel Santos,"Durante o semestre, pude observar um crescimen..."
4,Erika Pereira,"No semestre passado, tive a oportunidade de de..."


### definindo modelo de embedding na variável 'model'

In [124]:
# lista de modelos de embedding:

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

model = 'models/embedding-001'

models/embedding-001
models/text-embedding-004


In [130]:
# variável para embedding por CLUSTEIZAÇÃO - para gerar segmentar alunos por perfil vocacional

embedding_clustering = genai.embed_content(
    model = model,
    content = df['autoavaliacao'],
    task_type = 'CLUSTERING',
)

# variável para embedding por SIMILARIDADE - para: hierarquizar competências

embedding_similaridade = genai.embed_content(
    model = model,
    content = df['autoavaliacao'],
    task_type = 'SEMANTIC_SIMILARITY',
)

### funções embeddings

In [131]:
def fn_embedding_semantic_similarity(text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type='retrieval_query')['embedding']


In [132]:
df['Embeddings'] = df.apply(lambda row: fn_embedding_semantic_similarity(row['autoavaliacao']), axis=1)

In [133]:
df

,nome,autoavaliacao,Embeddings
0,Ana Silva,"No semestre passado, pude observar um crescime...","[0.028692823, -0.032471396, -0.0139682945, 0.0..."
1,Bruno Oliveira,"Ao longo do semestre, notei um progresso consi...","[-0.008241747, -0.03234989, -0.0201636, 0.0336..."
2,Carla Costa,"Neste semestre, tive a oportunidade de aprimor...","[0.013034234, -0.044011697, -0.0040123854, 0.0..."
3,Daniel Santos,"Durante o semestre, pude observar um crescimen...","[0.02348041, -0.02630075, -0.0015877283, 0.056..."
4,Erika Pereira,"No semestre passado, tive a oportunidade de de...","[0.011845743, -0.023316598, -0.019337894, 0.01..."


In [134]:
def consulta_similaridade_competencias(consulta, base, model):
  embedding_consulta_similaridade = genai.embed_content(model=model,
                                                        content=consulta,
                                                        task_type='SEMANTIC_SIMILARITY')

  produtos_escalares = np.dot(np.stack(df['Embeddings']), embedding_consulta_similaridade['embedding'])
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]['nome']



### Definir variáveis com as áreas do Conhecimento

In [142]:
# Linguagens e suas Tecnologias
linguas_tecnologias =  'livros, leitura, compreenção, linguagens, idiomas, culturas, comunicação, expressão, escrita, oralidade, gramática, vocabulário, tradução, interpretação, literatura, poesia, prosa, teatro, cinema, música, artes visuais, mídias digitais, internet, redes sociais, tecnologia da informação, informática, ferramentas digitais, aplicativos, plataformas online, globalização, multiculturalismo, interculturalidade, diversidade linguística, inclusão digital'

# Matemática e suas Tecnologias
matematica_tecnologias = 'números, operações matemáticas, cálculos, geometria, álgebra, trigonometria, estatística, probabilidade, lógica, raciocínio lógico, resolução de problemas, modelagem matemática, matemática financeira, matemática aplicada, física matemática, química matemática, biomatemática, economia matemática, ciência da computação, algoritmos, programação, software, hardware, internet, tecnologia da informação, informática, ferramentas digitais, aplicativos, plataformas online'

# Ciências Humanas e Sociais
humanas_sociais ='história, geografia, sociologia, filosofia, política, psicologia, antropologia, economia, direito, ética, cidadania, democracia, direitos humanos, justiça social, desigualdade social, movimentos sociais, globalização, multiculturalismo, interculturalidade, diversidade cultural, estudos de gênero, estudos étnico-raciais, estudos ambientais, sustentabilidade, desenvolvimento social, políticas públicas, relações internacionais'

# Ciências da Natureza e suas Tecnologias
naturais_tecnologias = 'física, química, biologia, geologia, astronomia, meio ambiente, ecologia, biodiversidade, sustentabilidade, desenvolvimento sustentável, recursos naturais, energia, água, ar, solo, clima, mudanças climáticas, poluição, saúde ambiental, educação ambiental, tecnologia ambiental, biotecnologia, nanotecnologia, robótica, inteligência artificial, internet das coisas, tecnologia da informação, informática, ferramentas digitais, aplicativos, plataformas online'

# Educação Física
educacao_fisica = 'jogos, atividades físicas, saúde física, saúde mental, bem-estar, qualidade de vida, movimento corporal, motricidade, desenvolvimento motor, coordenação motora, agilidade, força, resistência, flexibilidade, equilíbrio, postura, educação postural, ginástica, natação, atletismo, futebol, basquetebol, voleibol, handebol, tênis, judô, karatê, capoeira, dança, yoga, pilates, musculação, condicionamento físico, treinamento funcional, esportes radicais, lazer, recreação, tempo livre, cultura corporal, educação para o movimento, educação para a saúde, inclusão social, acessibilidade, diversidade corporal'

areas_conhecimento = [linguas_tecnologias, matematica_tecnologias, humanas_sociais, naturais_tecnologias, educacao_fisica]
#areas_conhecimento

## Realizar consulta de qual aluno foi destaque em cada área do conhecimento

In [140]:
consulta = 'livros, leitura, compreenção, linguagens, idiomas, culturas, comunicação, expressão, escrita, oralidade, gramática, vocabulário, tradução, interpretação, literatura, poesia, prosa, teatro, cinema, música, artes visuais, mídias digitais, internet, redes sociais, tecnologia da informação, informática, ferramentas digitais, aplicativos, plataformas online, globalização, multiculturalismo, interculturalidade, diversidade linguística, inclusão digital.'
resposta_final = consulta_similaridade_competencias(consulta, df, model)
print(resposta_final)

Carla Costa


### Definir quais alunos foram destaque em cada área do conhecimento:

In [153]:
for area in areas_conhecimento:
  consulta = area
  resposta_final = consulta_similaridade_competencias(consulta, df, model)
  print(f'Os alunos destaque foram: {resposta_final}') # Falta associar os nomes das áreas

Os alunos destaque foram: Carla Costa
Os alunos destaque foram: Ana Silva
Os alunos destaque foram: Erika Pereira
Os alunos destaque foram: Carla Costa
Os alunos destaque foram: Ana Silva
